In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys
from glob import glob
from hydra import initialize, compose
from matplotlib import colors
from pyHalo.preset_models import CDM
from tqdm import tqdm

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.utils import util
from mejiro.lenses import lens_util

In [2]:
pickles = glob(os.path.join(config.machine.dir_01, '01_hlwas_sim_detectable_lenses_sca*.pkl'))
scas = [int(f.split('_')[-1].split('.')[0][3:]) for f in pickles]
scas = sorted([str(sca).zfill(2) for sca in scas])
sca_dict = {}
total = 0
for sca in scas:
    pickle_path = os.path.join(config.machine.dir_01, f'01_hlwas_sim_detectable_lenses_sca{sca}.pkl')
    lens_list = util.unpickle(pickle_path)
    assert len(lens_list) != 0, f'No pickled lenses found. Check {pickle_path}.'
    sca_dict[sca] = lens_list
    total += len(lens_list)
count = total

In [3]:
for sca in scas:
    print(f'SCA {sca}: {len(sca_dict[sca])} lenses')

In [4]:
output_files = glob(config.machine.dir_00 + '/detectable_pop_*.csv')

uid = 0
dict_list = []
for sca_id in tqdm(scas):
    sca_id = str(sca_id).zfill(2)

    csvs = [f for f in output_files if f'sca{sca_id}' in f]

    if len(csvs) == 0:
        continue

    # get all runs associated with this SCA
    runs = [int(f.split('_')[-2]) for f in
            csvs]  # TODO I don't love string parsing that relies on file naming conventions

    lens_list = []
    for _, run in enumerate(runs):
        # unpickle the lenses from the population survey and create lens objects
        lens_paths = glob(
            config.machine.dir_00 + f'/run_{str(run).zfill(4)}_sca{sca_id}/detectable_lens_{str(run).zfill(4)}_*.pkl')

        if len(lens_paths) == 0:
            continue

        for _, lens_path in enumerate(lens_paths):
            lens_dict = util.unpickle(lens_path)
            dict_list.append(lens_dict)
            # lens = lens_util.unpickle_lens(lens, str(uid).zfill(8))
            # uid += 1
            # lens_list.append(lens)

0it [00:00, ?it/s]


In [7]:
sample = util.unpickle('/data/bwedig/mejiro/pipeline_dev/run_0017_sca18/detectable_lens_0017_sca18_00000.pkl')

from pprint import pprint

pprint(sample)

{'deflector_stellar_mass': 9900414985.46622,
 'deflector_velocity_dispersion': 177.21094480478806,
 'kwargs_model': {'cosmo': FlatLambdaCDM(name="Planck18", H0=67.66 km / (Mpc s), Om0=0.30966, Tcmb0=2.7255 K, Neff=3.046, m_nu=[0.   0.   0.06] eV, Ob0=0.04897),
                  'lens_light_model_list': ['SERSIC_ELLIPSE'],
                  'lens_model_list': ['SIE', 'SHEAR', 'CONVERGENCE'],
                  'lens_redshift_list': [1.0028630097305802,
                                         1.0028630097305802,
                                         1.0028630097305802],
                  'source_light_model_list': ['SERSIC_ELLIPSE'],
                  'source_redshift_list': [1.4340984967692338],
                  'z_source': 1.4340984967692338,
                  'z_source_convention': 6.0},
 'kwargs_params': {'kwargs_lens': [{'center_x': -0.03579819240710826,
                                    'center_y': 0.04357057874790285,
                                    'e1': -0.619215980465